# CS345 Project

## Team Members
1. Hamad Alyami
2. Benito Encarnacion

## Dataset
Our dataset was from Kaggle by a user called Mexwell. The data is paragraphs scraped from wikipedia in 2018 in 235 languages.

The dataset contains 235,000 datasets with balance between language proportions and a test and train split provided.

The downloaded folder from Kaggle contains:
- labels.csv: A file containing the language name, 2-3 letter code, German name, and language family of all the languages present in the dataset.
- README.txt: A file explaining the folder contents.
- urls.txt: A file containing the urls of where the paragraphs were found.
- x_test.txt: The testing data samples, paragraphs in multiple languages.
- x_train.txt: The training data samples, paragraphs in multiple langauges
- y_test.txt: The labels for the testing dataset, using the 2-3 letter codes found in labels.csv.
- y_train.txt: The labels for the training dataset, using the 2-3 letter codes found in labels.csv


## Project
Our project is to train and compare two ML models on the Latin Alphabet languages present in the dataset and compare their performance.

## Motivation
We decided to do this project because it allows us to explore practical applications of natural language processing and machine learning by working with real-world multilingual data. Language identification is an important task in many systems and applications like search engines, translation tools, and content moderation. Working with such a dataset gives us the opportunity to apply classification techniques in a meaningful way. By focusing on languages that use the Latin alphabet, we avoid complications from different writing systems while still working with a variety of languages.

## Models
The models we decided to work with in this project are:
- Multinomial Naive-Bayes (MNB): Uses word frequencies in each class, langauges in our case, to guess the most likely class for text it has not seen.

- Feed Forward Neural Network (FNN): An artificial Neural Network where information moves from input to output without looping back. It uses neurons, connected nodes, to learn patterns and make predictions.

### Data Preprocessing
We will begin by reading the data from the files then:
1. Remove Null Values
2. Filter to keep texts of languages we want using the 2-3 letter codes
3. Return both samples from x_test and x_train and labels from y_test and y_train stacked into X and y

In [65]:
#Understanding the data set
import pandas as pd
import numpy as np

def file_to_np_array(path, label):
    try:
        df = pd.read_csv(path, sep='<NonExistenceSeparator>', header=None, engine='python')
        print(f"{label}: Read!")
    except Exception as e:
        print(f"Error reading the {label} file: {e}")
        return None
    return df.to_numpy()


def clean_np_data(X, y):
    stacked = np.hstack((y, X)) # Stack y and X side by side
    # print(stacked.shape)
    clean_stacked = stacked[~np.any(pd.isna(stacked), axis=1), :] # Remove empty values
    # print(clean_stacked.shape)
    lang_codes = ['ita', 'fra', 'spa', 'por', 'eng', 'deu', 'nld', 'ind', 'fin', 'hau']
    true_clean = clean_stacked[np.isin(clean_stacked[:,0], lang_codes),:] # Remove all rows that aren't our target languages
    # print(true_clean.shape)
    return true_clean[:,1], true_clean[:,0] # Return cleaned as X and y split again

def clean_filter_and_stack(X_train_file, y_train_file, X_test_file, y_test_file):
    X_train_clean, y_train_clean = clean_np_data(file_to_np_array(X_train_file, X_train_file), 
                                       file_to_np_array(y_train_file, y_train_file))
    X_test_clean, y_test_clean = clean_np_data(file_to_np_array(X_test_file, X_test_file), 
                                       file_to_np_array(y_test_file, y_test_file))
    return np.hstack((X_train_clean, X_test_clean)), np.hstack((y_train_clean, y_test_clean))

X, y = clean_filter_and_stack("Data/x_train.txt", 
                                      "Data/y_train.txt", 
                                      "Data/x_test.txt", 
                                      "Data/y_test.txt")

print(X.shape, y.shape)

Data/x_train.txt: Read!
Data/y_train.txt: Read!
Data/x_test.txt: Read!
Data/y_test.txt: Read!
(10000,) (10000,)


In [77]:
def summarize_language_data(X, y):
    def count_words(text):
        return len(str(text).split())

    df = pd.DataFrame({'text': X.flatten(), 'lang': y.flatten()})
    df['word_count'] = df['text'].apply(count_words)

    total_count = len(df)
    grouped = df.groupby('lang').agg(
        entry_count=('lang', 'count'),
        avg_word_count=('word_count', 'mean')
    ).reset_index()

    grouped['percentage'] = (grouped['entry_count'] / total_count) * 100
    grouped = grouped.drop(columns='entry_count')

    return grouped

display(summarize_language_data(X, y))


,lang,avg_word_count,percentage
0,deu,59.762,10.0
1,eng,70.455,10.0
2,fin,48.431,10.0
3,fra,67.707,10.0
4,hau,75.802,10.0
5,ind,57.147,10.0
6,ita,68.192,10.0
7,nld,55.657,10.0
8,por,66.184,10.0
9,spa,67.295,10.0


#### Data Split
Here we use Sklearn train_test_split to split our data into 70/30 train and test splits, respectively, after shuffling them randomly.

In [67]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(7000,) (7000,)
(3000,) (3000,)


In [68]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(strip_accents='unicode')
X_tr_vectors = vectorizer.fit_transform(X_train)
X_te_vectors = vectorizer.transform(X_test)
print("Done vectorizing")

Done vectorizing


In [69]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_tr_vectors, y_train)
print("Done training MNB")

Done training MNB


In [70]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_te_vectors)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)
# print(y_pred[0:10])
# print(X_test[0:10])

# def group_accuracy(group):
#     return accuracy_score(group['data'], group['pred'])

# df = pd.DataFrame({'data': y_test, 'pred': y_pred})

# lang_accuracy = df.groupby('data').apply(group_accuracy).reset_index(name='accuracy')

# print("\nAccuracy per language:")
# print(lang_accuracy)



Accuracy:  0.981


In [74]:
df = pd.DataFrame({'true_label': y_test, 'pred': y_pred})
df['correct'] = (df['true_label'] == df['pred']).astype(int)

lang_accuracy = df.groupby('true_label')['correct'].mean()

print("\nAccuracy per language:")
print(lang_accuracy)


Accuracy per language:
true_label
deu    0.975694
eng    1.000000
fin    0.993151
fra    0.993355
hau    0.996622
ind    0.986159
ita    0.968553
nld    0.982759
por    0.937282
spa    0.976471
Name: correct, dtype: float64


In [75]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import make_pipeline

model = make_pipeline(CountVectorizer(), MultinomialNB())

num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=17)

cross_val_results = cross_val_score(model, X.flatten(), y.flatten(), cv=kf)

all_preds = []
all_true = []

print("Cross-Validation Results (Accuracy):")
for i, result in enumerate(cross_val_results, 1):
    print(f"  Fold {i}: {result * 100:.2f}%")

    all_preds.extend(y_pred)
    all_true.extend(y_test)

print(f'Mean Accuracy: {cross_val_results.mean() * 100:.2f}%')

df = pd.DataFrame({'true': all_true, 'pred': all_preds})
df['correct'] = (df['true'] == df['pred']).astype(int)

# Group by language and compute mean accuracy across all folds
lang_accuracy = df.groupby('true')['correct'].mean()

print("\nMean Accuracy per Language Across All Folds:")
print(lang_accuracy)

Cross-Validation Results (Accuracy):
  Fold 1: 98.25%
  Fold 2: 98.30%
  Fold 3: 98.25%
  Fold 4: 98.60%
  Fold 5: 98.40%
Mean Accuracy: 98.36%

Mean Accuracy per Language Across All Folds:
true
deu    0.975694
eng    1.000000
fin    0.993151
fra    0.993355
hau    0.996622
ind    0.986159
ita    0.968553
nld    0.982759
por    0.937282
spa    0.976471
Name: correct, dtype: float64


In [86]:
# Reorganizing data for Feed Forward Neural Network input

# 1) convert string labels into integer labels
#      and make func to convert back

all_str_labels = ['ita', 'fra', 'spa', 'por', 'eng', 'deu', 'nld', 'ind', 'fin', 'hau']

def str_labels_to_int_labels(labelArr, string_labels):
    rtn = np.empty(labelArr.shape, dtype=int)
    for i, v in enumerate(string_labels):
        rtn[labelArr == v] = i
    return rtn

def int_labels_to_str_labels(labelArr, string_labels):
    rtn = np.empty(labelArr.shape, dtype='object')
    for i, v in enumerate(string_labels):
        rtn[labelArr == i] = v
    return rtn

# print(y_test[0:5])
# y1 = str_labels_to_int_labels(y_test, all_str_labels)
# print(y1[0:5])
# y2 = int_labels_to_str_labels(y1, all_str_labels)
# print(y2[0:5])

In [111]:
# 2) convert data into multi-column matrix of characters
#      and make func to convert back

def str_vec_to_float_matrix(strVec, longest_str_len):
    # Pad strings to all be equal length
    padded_strVec = np.char.ljust(strVec, longest_str_len, fillchar=' ')

    # turn vector of strings into matrix of characters
    stacked_char_matrix = np.vstack([np.array(list(s)) for s in padded_strVec])

    # turn char matrix into int matrix
    char_matrix_to_int_matrix = np.vectorize(ord)
    int_matrix = char_matrix_to_int_matrix(stacked_char_matrix)

    #normalize and scale so each value is a float between 0 and 1
    matrix_max = np.max(int_matrix)
    matrix_min = np.min(int_matrix)
    min_subtracted_matrix = int_matrix - matrix_min
    normalized_matrix = (min_subtracted_matrix / (matrix_max - matrix_min))
    return normalized_matrix

# Don't need to convert matrices back into rows of text because the neural network isn't designed to generate anything, just classify
# def int_matrix_to_str_vec(intMatrix):
#     int_matrix_to_char_matrix = np.vectorize(chr)
#     char_matrix = int_matrix_to_char_matrix(intMatrix)
#     padded_strVec = np.array(["".join(r) for r in char_matrix])
#     return np.char.rstrip(padded_strVec)

# print(X_test[0])
# X_x = str_vec_to_int_matrix(X_test.astype(str))
# print(X_x[0])
# X_y = int_matrix_to_str_vec(X_x)
# print(X_y[0])

In [112]:
# 3) use them both

def convert_to_FFNN_format(Xr, Xe, yr, ye):
    all_string_labels = ['ita', 'fra', 'spa', 'por', 'eng', 'deu', 'nld', 'ind', 'fin', 'hau']
    max_str_len_1 = np.max(np.char.str_len(Xr.astype(str)))
    max_str_len_2 = np.max(np.char.str_len(Xe.astype(str)))
    max_str_len = max(max_str_len_1, max_str_len_2)
    Xr_rtn = str_vec_to_float_matrix(Xr.astype(str), max_str_len)
    Xe_rtn = str_vec_to_float_matrix(Xe.astype(str), max_str_len)
    return (Xr_rtn, 
            Xe_rtn,
            str_labels_to_int_labels(yr, all_string_labels), 
            str_labels_to_int_labels(ye, all_string_labels))

(X_tr_nn, X_te_nn, y_tr_nn, y_te_nn) = convert_to_FFNN_format(X_train, X_test, y_train, y_test)

print("Done converting data into FFNN format")

Done converting data into FFNN format


In [113]:
# print(X_tr_nn[0])
# print(X_tr_nn[1])
print(X_tr_nn.shape)
print(X_te_nn.shape)

(7000, 5577)
(3000, 5577)


In [114]:
# Applying properly structured data to a basic FFNN

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy

print("Done importing Tensorflow Stuff")

Done importing Tensorflow Stuff


In [130]:
# Make FFNN

sample_length = X_tr_nn[0].shape[0]

FFNN_model = Sequential([
    Dense(sample_length, activation='relu'),
    Dense(sample_length, activation='relu'),
    Dense(1024, activation='relu'),
    Dense(1024, activation='relu'),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

FFNN_model.compile(optimizer=Adam(),
                   loss=SparseCategoricalCrossentropy(), 
                   metrics=[SparseCategoricalAccuracy()])

In [131]:

for i in range(100):
    FFNN_model.fit(X_tr_nn, y_tr_nn)
    test_loss, test_acc = FFNN_model.evaluate(X_te_nn, y_te_nn)
    print(f'\nTest accuracy: {test_acc}')

94/94 [==============================] - 3s 27ms/step - loss: 2.3033 - sparse_categorical_accuracy: 0.0997

Test accuracy: 0.09966666996479034
94/94 [==============================] - 2s 26ms/step - loss: 2.3035 - sparse_categorical_accuracy: 0.0957

Test accuracy: 0.09566666930913925
94/94 [==============================] - 3s 37ms/step - loss: 2.3044 - sparse_categorical_accuracy: 0.0960

Test accuracy: 0.09600000083446503
94/94 [==============================] - 3s 30ms/step - loss: 2.3039 - sparse_categorical_accuracy: 0.0960

Test accuracy: 0.09600000083446503
94/94 [==============================] - 3s 32ms/step - loss: 2.3053 - sparse_categorical_accuracy: 0.0957

Test accuracy: 0.09566666930913925
94/94 [==============================] - 3s 32ms/step - loss: 2.3060 - sparse_categorical_accuracy: 0.0970

Test accuracy: 0.09700000286102295
94/94 [==============================] - 3s 31ms/step - loss: 2.3158 - sparse_categorical_accuracy: 0.0970

Test accuracy: 0.09700000286102295

KeyboardInterrupt: 